Below we install the Python library by Microsoft, `dowhy`. It can learn causal graphs from data and carry out do-calculus derivations to find ways of using.

We also import some additional libraries we'll be using.

In [ ]:
!pip install dowhy
 
import os, sys
sys.path.append(os.path.abspath("../../"))
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 5.8MB 8.3MB/s 
  Found existing installation: pydot 1.3.0
    Uninstalling pydot-1.3.0:
      Successfully uninstalled pydot-1.3.0
  Found existing installation: sympy 1.1.1
    Uninstalling sympy-1.1.1:
      Successfully uninstalled sympy-1.1.1


A clinical trial in the 1980s called the Infant Health and Development Program or IHDP looked at the cognitive capacity of premature infants and the impact of treatments on their development. The IHDP dataset contains instances from their randomized study and records properties of the children and their caregivers. An important question the data can address is --- does treatment from a specialized therapist result in better outcomes than treatment from other care givers?

The data has a bunch of features. “Treatment” is a 0/1 variable that indicates the absence or presence of specialized treatment for each child. The feature labeled “y_factual” is an assessment of the child’s improvements in cognitive development.

The other features are not given semantically meaningful names as a way of protecting the privacy of the participants in the study.

We download and process the `data` below.

In [ ]:
# read the IHDP dataset.
 
data= pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
 
for i in range(1,26):
    col.append("x"+str(i))
data.columns = col
data = data.astype({"treatment": bool})
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,True,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,False,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,False,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,False,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,False,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


We create a `CausalModel` to process the data using the `data`, `treatement`, and `y_factual` `outcome`.  We call `identify_effect` on the `model` to derive a causal effect.

Note that when you run this routine, the code reminds you that it’s making some educated guesses about the way that the unobserved confounders can impact the model. The `dowhy` software steers users away from taking the results at face value, and into looking more closely at possible causal effects.

In [ ]:
# Create a causal model from the data and the "x" variables as common causes.
xs = ""
for i in range(1,26):
    xs += ("x"+str(i)+"+")
 
model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=xs.split('+')
        )
 
#Identify the causal effect
identified_estimand = model.identify_effect()

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['y_factual']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['', 'x8', 'x4', 'x11', 'x5', 'x3', 'x25', 'x18', 'x10', 'x2', 'x16', 'x13', 'x23', 'x20', 'x6', 'x19', 'x12', 'x21', 'x7', 'x22', 'x17', 'x1', 'x15', 'x9', 'x14', 'x24']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


We estimate the effect of the treatment on the outcomes in two ways by calculating the “average treatment effect” or `ATE`. That’s a correlational measure of treatment and outcomes.

To estimate the average treatment effect, we separatethe instances where the treatment is given, `data_1`, from the instances where the treatment was NOT given, `data_0`. The `ATE` is the difference between the means in these two sets.


In [ ]:
data_1 = data[data["treatment"]==1]
data_0 = data[data["treatment"]==0]
print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

ATE 4.021121012430832


We also use Dowhy’s `estimate_effect` function to more precisely characterize the treatment effect. We use `backdoor_propensity_score_weighting`, which uses the do-calculus to re-assess how much of those gains are really attributable to the treatment and not other factors.

In [ ]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.propensity_score_weighting")

print("Causal Estimate is " + str(estimate.value))

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x8+x4+x11+x5+x3+x25+x18+x10+x2+x16+x13+x23+x20+x6+x19+x12+x21+x7+x22+x17+x1+x15+x9+x14+x24
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Causal Estimate is 3.409737824408194
